In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F 
from pyspark.sql.types import *
from delta import *

In [ ]:
builder = SparkSession.builder.appName("olist_demo") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .config("spark.sql.repl.eagerEval.enabled", "true") \
    .config("spark.executor.memory", "2g")\
    .config("spark.driver.memory", "4g")\
    .config("spark.driver.memory", "4g")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

## Config

In [3]:
environment = '' # dev;uat;prd

In [ ]:
base_path = '../../../data'  # This will be overwritten by Papermill

In [4]:
path = {
    'slv': {'products': f'{base_path}/silver/mtd/mtd_products'}, 
    'snk': f'{base_path}/gold/dim/dim_products'
}

## Load

In [5]:
df = spark.read.format('delta').load(path['slv']['products'])

In [6]:
df_snk = df.select(
    F.col('id_product'),
    F.col('nm_product_category'),
    F.col('nm_product_category_english'),
    F.col('nm_product_lenght'),
    F.col('ds_product_lenght'),
    F.col('qt_product_photos'),
    F.col('qt_product_weight_g'),
    F.col('qt_product_length_cm'),
    F.col('qt_product_height_cm'),
    F.col('qt_product_width_cm'),
)

In [7]:
df_snk = df_snk.withColumn("dh_insert_gld", F.current_timestamp())

## Sink

In [ ]:
df_snk.write.format('delta')\
.mode(saveMode='overwrite')\
.save(path['snk'])